# Load libraries

In [ ]:
!pip install neuron

In [ ]:
from neuron import h
from neuron.units import mV, ms
h.load_file('stdrun.hoc')

1.0

In [ ]:
from bokeh.io import output_notebook
import bokeh.plotting as plt
output_notebook()

# Setup the biophysical model

In [ ]:
axon = h.Section(name='axon')
axon.insert(h.hh)

axon

In [ ]:
axon.L

100.0

# Add a synapse

In [ ]:
syn = h.ExpSyn(axon(0))
syn.tau = 1 * ms
syn.e = 0 * mV

#Add a stimulus

In [ ]:
stim = h.NetStim()
stim.number = 9999999
stim.interval = 10 * ms
stim.noise = True
stim.start = 0 * ms

stim_times = h.Vector()

# Connect stimulus to synapse

In [ ]:
nc = h.NetCon(stim, syn)
nc.delay = 0 * ms
nc.weight[0] = 0.2
nc.record(stim_times)

0.0

# Setup timeseries recording

In [ ]:
t = h.Vector().record(h._ref_t)
v = h.Vector().record(axon(0.5)._ref_v)

# Setup spike event recording

In [ ]:
spike_data = None  # starting with an invalid value will allow us to catch if 
                   # not initializing correctly

## Make sure it starts out empty

In [ ]:
def reset_spike_data():
  global spike_data
  spike_data = []

reset_spike_data_fih = h.FInitializeHandler(reset_spike_data)

## Now for when there is an event

In [ ]:
def store_spike_data():
  spike_data.append({
      "t": h.t,
      "m": axon(0.5).hh.m,
      "n": axon(0.5).hh.n,
      "h": axon(0.5).hh.h,
      "v": axon(0.5).v,
      "syn_i": syn.i
  })

# the NetCon connection
nc_self = h.NetCon(axon(0.5)._ref_v, None, sec=axon)
nc_self.threshold = 0 * mV
nc_self.record(store_spike_data)


0.0

# Run the simulation

In [ ]:
h.finitialize(-65 * mV)
h.continuerun(100 * ms)

0.0

# Plot it

In [ ]:
f = plt.figure(x_axis_label='t (ms)', y_axis_label='v (mV)')
f.line(t, v, line_width=2)
plt.show(f)

# spike_data is a list of dictionaries, one for each spike

In [ ]:
spike_data

[{'h': 0.4119555356327057,
  'm': 0.5887316210713128,
  'n': 0.42212343313885425,
  'syn_i': -0.14366184230437265,
  't': 7.27500000010007,
  'v': 3.9089321692200087},
 {'h': 0.35104734497980616,
  'm': 0.6258906056236117,
  'n': 0.4579642845718529,
  'syn_i': -0.01028207701574703,
  't': 29.675000000098848,
  'v': 5.215569708787357},
 {'h': 0.41475896980600185,
  'm': 0.5702579789520994,
  'n': 0.4167180461500577,
  'syn_i': -0.2908150450059131,
  't': 78.1500000001092,
  'v': 1.3171200516227266}]